In [3]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

# Imports

In [4]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [5]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path

In [6]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [7]:
from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from scipy.signal import savgol_filter

In [180]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run ../../Functions/1.2/data_fct_Adrien_point_by_point

matplotlib.rcParams.update({'font.size': 25})

In [9]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Sélection fichier

In [10]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try00
> Filename : Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [0, 5, 12, 21, 30, 39, 49]
> slashcond | 5
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 2800


In [11]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC01_RF08', 'DC01_RF09', 'DC01_RF10', 'DC01_RF11', 'DC03_RF08', 'DC03_RF09', 'DC03_RF10', 'DC03_RF11', 'DC05_RF08', 'DC05_RF09', 'DC05_RF10', 'DC05_RF11', 'DC07_RF08', 'DC07_RF09', 'DC07_RF10', 'DC07_RF11', 'DC09_RF08', 'DC09_RF09', 'DC09_RF10', 'DC09_RF11', 'DC11_RF08', 'DC11_RF09', 'DC11_RF10', 'DC11_RF11', 'DC18_RF08', 'DC18_RF09', 'DC18_RF10', 'DC18_RF11']
000 > DC01_RF08
001 > DC01_RF09
002 > DC01_RF10
003 > DC01_RF11
004 > DC03_RF08
005 > DC03_RF09
006 > DC03_RF10
007 > DC03_RF11
008 > DC05_RF08
009 > DC05_RF09
010 > DC05_RF10
011 > DC05_RF11
012 > DC07_RF08
013 > DC07_RF09
014 > DC07_RF10
015 > DC07_RF11
016 > DC09_RF08
017 > DC09_RF09
018 > DC09_RF10
019 > DC09_RF11
020 > DC11_RF08
021 > DC11_RF09
022 > DC11_RF10
023 > DC11_RF11
024 > DC18_RF08
025 > DC18_RF09
026 > DC18_RF10
027 > DC18_RF11
> condition names ['DC', 'RF']
> number of points 28
> N_ions = 1024
> e_GMol = 50


In [12]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir,points_and_coord,condition_parameters,slash_cfg) # stopping_power  ,forcenumtry=28

data0 = data_name[0]
Tvar = PMandT[1]

> Points | 28
> Simulations pour chaque point | ['Try00', 'Try01', 'Try02', 'Try03', 'Try04', 'Try05', 'Try06', 'Try07', 'Try08', 'Try09', 'Try10', 'Try11', 'Try12', 'Try13', 'Try14', 'Try15', 'Try16', 'Try17', 'Try18', 'Try19', 'Try20', 'Try21', 'Try22', 'Try23', 'Try24', 'Try25', 'Try26', 'Try27', 'Try28', 'Try29', 'Try30', 'Try31', 'Try32', 'Try33', 'Try34', 'Try35', 'Try36', 'Try37', 'Try38', 'Try39', 'Try40', 'Try41', 'Try42', 'Try43', 'Try44', 'Try45', 'Try46', 'Try47', 'Try48', 'Try49', 'Try50', 'Try51', 'Try52', 'Try53', 'Try54', 'Try55', 'Try56', 'Try57', 'Try58', 'Try59', 'Try60', 'Try61', 'Try62', 'Try63', 'Try64', 'Try65', 'Try66', 'Try67', 'Try68', 'Try69', 'Try70', 'Try71', 'Try72', 'Try73', 'Try74', 'Try75', 'Try76', 'Try77', 'Try78', 'Try79', 'Try80', 'Try81', 'Try82', 'Try83', 'Try84', 'Try85', 'Try86', 'Try87', 'Try88', 'Try89', 'Try90', 'Try91', 'Try92', 'Try93', 'Try94', 'Try95', 'Try96', 'Try97', 'Try98', 'Try99']
Hello
Point n° 0
00 - 00  >  /home/adrian/RemoteFS/

00 - 58  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try58/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 59  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try59/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 60  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try60/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 61  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try61/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 62  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try62/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 63  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try63/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 64  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try64/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 65  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try65/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_

01 - 23  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try23/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG
01 - 24  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try24/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG
01 - 25  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try25/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG
01 - 26  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try26/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG
01 - 27  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try27/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG
01 - 28  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try28/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG
01 - 29  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try29/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG
01 - 30  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try30/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_

01 - 89  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try89/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG
01 - 90  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try90/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG
01 - 91  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try91/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG
01 - 92  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try92/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG
01 - 93  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try93/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG
01 - 94  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try94/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG
01 - 95  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try95/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG
01 - 96  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try96/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_

02 - 54  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try54/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 55  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try55/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 56  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try56/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 57  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try57/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 58  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try58/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 59  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try59/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 60  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try60/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 61  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try61/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_

...

24 - 76  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try76/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
24 - 77  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try77/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
24 - 78  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try78/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
24 - 79  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try79/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
24 - 80  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try80/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
24 - 81  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try81/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
24 - 82  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try82/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
24 - 83  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try83/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_

25 - 41  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try41/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_S1.0RFG
25 - 42  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try42/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_S1.0RFG
25 - 43  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try43/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_S1.0RFG
25 - 44  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try44/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_S1.0RFG
25 - 45  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try45/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_S1.0RFG
25 - 46  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try46/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_S1.0RFG
25 - 47  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try47/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_S1.0RFG
25 - 48  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try48/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_

26 - 06  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try06/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
26 - 07  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try07/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
26 - 08  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try08/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
26 - 09  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try09/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
26 - 10  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try10/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
26 - 11  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try11/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
26 - 12  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try12/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
26 - 13  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try13/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_

26 - 72  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try72/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
26 - 73  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try73/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
26 - 74  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try74/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
26 - 75  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try75/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
26 - 76  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try76/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
26 - 77  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try77/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
26 - 78  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try78/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_S1.0RFG
26 - 79  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF10/Try79/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.6000D+01V_D1.0_

27 - 37  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try37/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 38  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try38/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 39  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try39/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 40  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try40/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 41  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try41/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 42  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try42/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 43  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try43/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 44  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try44/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_

In [13]:
# Automatic retrieve of GMol energy
onlyfiles = [f for f in listdir(all_subdir[0]) if isfile(join(all_subdir[0], f)) and "SimuType" in f and ".dat" in f]
aux_energie = onlyfiles[1].find('0RFG')
aux_energie_unit = onlyfiles[1].find('eV')
E0_GMol = onlyfiles[1][aux_energie+4:aux_energie_unit+2]
E0_GMol = re.sub('k','000',E0_GMol)
print(E0_GMol)
E0_GMol = int(E0_GMol.strip('eV'))
print(E0_GMol)

50eV
50


# Plots
Nuage d'ions, Temp et Pos

## Load temperature from Langevin files

In [549]:
## Data loading with hardcode names
eta = 1.5e-20

plot_point = 10
plot_try = 28
# str_eta_chiffre    = float(f'{eta[plot_point]:1.2e}'[:4])/10
# str_eta_puissance  = int(re.sub('e','D',f'{eta[plot_point]:1.2e}'[6:]))-1
# str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
str_eta = '0.15D-19'
dirname_hc = str(data0[plot_point][plot_try])+'/'
# Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
data0_hc    = '/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data4_hc    = '/Temp_SimuType4_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data2_hc    = '/Temp_SimuType2_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'

print(dirname_hc)
print(datainit_hc)

/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try28/
Temp_3D_Harmo_N1024_T500uK_F0.15D-19Kg_s_5


In [550]:
N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T =\
load_Temp_init_bin_Lan(dirname_hc+datainit_hc,1)

len save_T [55440]
[4] [4] [8] [8] [8] [55440]
N_ions 1024
j_save 990
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.0005]
[5.05000000e-07 1.80803062e-05 9.71782985e-04 2.24273998e-05
 4.42758341e+01 1.32873622e+00 6.48707834e+01]


In [551]:
# Get T and fluo PM  All SimuType
onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and "SimuType" in f and ".dat" in f]
print(sort(onlyfiles))
only_init = [f for f in onlyfiles if "SimuTypeQ" in f]
ta, te, ti = plot_T_and_PM_InitQ_Inje_Evol(dirname_hc,only_init[0].strip('.dat')[4:],flag_plot=1,fig_name='0',xlim1=(0,11),ylim1=(2e-4,500),ylim2=(2,65))
# tt2, T_CM2, T_aux2, PM2
# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)
print(dirname_hc,only_init[0].strip('.dat')[4:])

['Temp_SimuType2_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG.dat'
 'xva_SimuType4_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV.dat']
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try28/ _SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG


In [552]:
# recover duration for each step of simu (i_free__fly, i_laser_fly ...)

address = all_subdir[plot_point]
print(address)
info_files = sort( [f for f in listdir(address) if isfile(join(address, f)) and ".info" in f] )
print(sort(info_files))


tmp_info = loadtxt(data0[plot_point][plot_try]+'/'+info_files[0],encoding ='utf-8',
            skiprows=1, usecols=(0),
            unpack=True)
eta = tmp_info[-2]
i_free__fly_Lan = tmp_info[-1]

tmp_info = loadtxt(data0[0][0]+'/'+'xva_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.info',encoding ='utf-8',
            skiprows=0, usecols=(0),
            unpack=True)
i_laser_fly_quench = tmp_info[-2]
i_laser_fly = tmp_info[-1]

n_dt = 100
dt = 1/2e6

print(eta)
print(i_free__fly_Lan)
print(i_laser_fly_quench)
print(i_laser_fly)

/home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try10
['Langevin_cooling.info'
 'xva_SimuType2_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG50eV.info'
 'xva_SimuType4_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG50eV.info'
 'xva_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.info']
1.5e-20
1000.0
8000.0
6000.0


In [553]:
# Plot temperature Langevin + md laser  ALL SimuType   ALL TEMP SEPARATED

tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
tt_md = ta[0]
T_aux = te[0]

tt = concatenate((tt_Lan,tt_md))
T_all = concatenate((save_T[:,4:],T_aux),axis=0)

PM = ti[0]

#fig_name = file_name[-9:]
figname = 'T_and_PM_plot_example'
fig1 = plt.figure(num=figname,clear='True')
fig1.set_size_inches(11.69,8.27)
# set height ratios for sublots
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
# the fisrt subplot
ax1 = plt.subplot(gs[0])

ax1.semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
ax1.semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
ax1.semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
#semilogy(tt_Lan*1.e3,save_T[:,0], label='Tx',color='C0')
#semilogy(tt_Lan*1.e3,save_T[:,1], label='Ty',color='C1')
#semilogy(tt_Lan*1.e3,save_T[:,2], label='Tz',color='C2')
ax1.vlines(ta[1]*1e3,1e-4,1e4,color='red')
ax1.vlines(ta[2]*1e3,1e-4,1e4,color='red')
time_switch = i_free__fly_Lan*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='green',linestyle='--')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='orange',linestyle='-')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
ax1.hlines(0.5,7.5,8.5, linestyle=':')
# ax1.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
ax1.grid()
ylabel('T [K]')

# annotate('Laser ON', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul')
,
# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

# ax2 = subplot(212,sharex=ax1)
ax2 = plt.subplot(gs[1], sharex = ax1)
plot(tt_md*1e3,PM, label='Tx')
time_switch = i_free__fly_Lan*dt
ax2.vlines(time_switch*1e3,1e-4,70,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax2.vlines(time_switch*1e3,1e-4,70,color='green',linestyle='--')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax2.vlines(time_switch*1e3,1e-4,70,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
# ax2.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,90,color='purple')
ax2.grid()
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

xlabel('time [ms]')
ylabel('cts/500ns')

left, bottom, width, height = [0.25, 0.56, 0.39, 0.38] # cadre zoom
axins = fig1.add_axes([left, bottom, width, height])

axins.semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
axins.semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
axins.semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
axins.hlines(0.5,7.4,7.66, linestyle=':')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
axins.vlines(time_switch*1e3,1e-4,1e4,color='orange',linestyle='-')
time_switch = (i_free__fly_Lan+i_laser_fly  +i_laser_fly_quench)*dt
axins.vlines(ta[2]*1e3,1e-4,1e4,color='red')

x1, x2, y1, y2 = 7.48, 7.65, 0.0002, 50 # limites zoom
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.yaxis.set_label_position("right")
axins.yaxis.tick_right()


locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
axins.yaxis.set_major_locator(locmaj)
locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
axins.yaxis.set_minor_locator(locmin)
axins.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

axins.grid(b=True, which='major', linestyle='-')
axins.grid(b=True, which='minor', linestyle='--')

# axins.tick_params(axis="y",direction="in", pad=-53)
mark_inset(ax1, axins, loc1=1, loc2=3, fc="none", ec="0.25")
# axins.invert_yaxis()
# plt.setp(axins.get_xticklabels(), visible=False)
# plt.setp(axins.get_yticklabels(), fontsize=10)

ax1.set_xlim(-0.15, 10.75)
ax1.set_ylim(0.0001, 1200)

plt.tight_layout()
subplots_adjust(hspace=0.0)
# savefig(fname = figname+'_'+re.sub('/','_',dirname_hc[31:-1])+'.png',dpi=300)
# savefig(fname+'_D'+str(Detuning)+'_50eV'+'.png',dpi=300)

/home/adrian/.local/lib/python3.6/site-packages/matplotlib/figure.py:2267: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


In [554]:
final_tt = tt[-1]*1000
final_tt = float(str(final_tt)[0:5])
pts_to_1ms = round(len(tt)/final_tt) # 2 coups/µs * 1000 µs/ms
PM_start_offset = pts_to_1ms*tt[-len(PM)]*1000
index_ms = []
PM_1ms = []
tt_1ms_step = [2,3,4,5,6,7,8,9]
detec_eff = 1e-3
for k in tt_1ms_step:
    index_ms_tmp = int(floor(k/final_tt*len(tt)) ) - int(round(PM_start_offset))
    index_ms.append(index_ms_tmp)
    PM_1ms.append(sum( PM[index_ms_tmp-pts_to_1ms//2:index_ms_tmp+pts_to_1ms//2] )*detec_eff)

final_tt = tt[-1]*1000
final_tt = float(str(final_tt)[0:5])
pts_to_1us = 2 # 2 coups/µs
index_us = []
PM_1us = []
tt_1us_step = tt[1::2]+i_free__fly_Lan*dt
for k in range(1,2*len(tt_1us_step),2):
    PM_1us.append(sum(PM[k:k+2]))

In [555]:
gmol_address = os.path.join(all_subdir[100*10+28],'trj_SimuType4_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV')
tgmol, r_x,r_y,r_z, v_x,v_y,v_z, a_x,a_y,a_z = load_trj(gmol_address)
t_in = 0
t_out = 0
for j,k in enumerate(r_z):
    if k*1000 < -dim_nu[10,0,2]/2:
        t_in = tgmol[j]
    if k*1000 < dim_nu[10,0,2]/2:
        t_out = tgmol[j]
print(t_in*1000,t_out*1000)

7.5132625350080575 7.515785026057564


In [583]:
PM_1ms[-3],mean(PM[-2000:])*2

(72.16198999999999, 12.300139999999999)

In [584]:
PM_1ms[-3]/mean(PM[-2000:])/2

5.866761679135359

In [593]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Plot temperature Langevin + md laser  ALL SimuType  ALL TEMP MEAN
matplotlib.rcParams.update({'font.size': 25})

tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
tt_md = ta[0]
T_aux = te[0]

tt = concatenate((tt_Lan,tt_md))
T_all = concatenate((save_T[:,4:],T_aux),axis=0)

PM = ti[0]

#fig_name = file_name[-9:]
figname = 'T_and_PM_plot_example'
fig1 = plt.figure(num=figname,clear='True')
fig1.set_size_inches(11.69,8.27)
# set height ratios for sublots
gs = gridspec.GridSpec(2, 1, height_ratios=[7, 3]) 
# the fisrt subplot
ax1 = plt.subplot(gs[0])

ax1.semilogy(tt*1e3,savgol_filter(mean(T_all,axis=1),25,1), label='Tx',color='red',lw=2.5)
#semilogy(tt_Lan*1.e3,save_T[:,0], label='Tx',color='C0')
#semilogy(tt_Lan*1.e3,save_T[:,1], label='Ty',color='C1')
#semilogy(tt_Lan*1.e3,save_T[:,2], label='Tz',color='C2')
# ax1.vlines(ta[1]*1e3,1e-4,1e4,color='red')
# ax1.vlines(ta[2]*1e3,1e-4,1e4,color='red')
time_switch = i_free__fly_Lan*dt
ax1.vlines(time_switch*1e3,1e-5,1e4,color='black')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax1.vlines(time_switch*1e3,1e-5,1e4,color='black',linestyle='--')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
# ax1.vlines(time_switch*1e3,1e-4,1e4,color='black',linestyle='-')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
# ax1.vlines((ta[2])*1e3,1e-4,1e4,color='black')
# ax1.hlines(0.5,7.5,8.5, linestyle=':')
# ax1.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
ax1.grid()
ylabel('T [K]')

ax1.annotate('1', xy=(0.22,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('2', xy=(2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('3', xy=((3.5+7.5)/2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
# ax1.annotate('4', xy=((7.5+7.57)/2,2.5e-4), xycoords='data',
# size=15, ha='center', va='top', color='xkcd:black',
# bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('5', xy=((7.5+10.5)/2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))

# annotate('Laser ON', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul')

# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

# ax2 = subplot(212,sharex=ax1)
ax2 = plt.subplot(gs[1], sharex = ax1)

# ax2.plot(tt_md*1e3,PM, label='Tx')
ax2.scatter(tt_1ms_step,PM_1ms, label='cts/ms',marker='P',s=200,lw=1.5,color='black',zorder=100)
ax2.plot(tt_1us_step[:-500]*1e3,PM_1us[:-500], label='cts/µs',color='blue')
ax2.legend()
time_switch = i_free__fly_Lan*dt
ax2.vlines(time_switch*1e3,1e-4,150,color='black')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
# ax2.vlines(time_switch*1e3,1e-4,150,color='black',linestyle='--')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
# ax2.vlines(time_switch*1e3,1e-4,70,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
# ax2.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,90,color='purple')
ax2.grid()
# ax2.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=20))
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

xlabel('time [ms]')
ylabel('PM signal')

left, bottom, width, height = [0.345, 0.565, 0.35, 0.38] # cadre zoom
axins = fig1.add_axes([left, bottom, width, height])

axins.semilogy(tt*1e3,mean(T_all,axis=1), label='Tx',color='red',lw=2.5,marker='x')
# axins.hlines(0.5,7.4,7.66, linestyle=':')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
axins.vlines(time_switch*1e3,1e-4,1e4,color='black',linestyle='-')
time_switch = (i_free__fly_Lan+i_laser_fly  +i_laser_fly_quench)*dt
axins.vlines(ta[2]*1e3,1e-4,1e4,color='black')

axins.add_patch(
     patches.Rectangle(
        (t_in*1000, 3e-4),
        (t_out - t_in)*1000,
        0.1,
        edgecolor ='k',
        facecolor =  cc('grey',0.5),
        fill=True
     ) )

x1, x2, y1, y2 = 7.44, 7.63, 0.00012, 50 # limites zoom
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

# axins.yaxis.tick_right()


locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
axins.yaxis.set_major_locator(locmaj)
locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
axins.yaxis.set_minor_locator(locmin)
axins.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
plt.setp(axins.get_xticklabels(), backgroundcolor="w")
plt.setp(axins.get_yticklabels(), backgroundcolor="w")

axins.grid(b=True, which='major', linestyle='-')
axins.grid(b=True, which='minor', linestyle='--')

# axins.tick_params(axis="y",direction="in", pad=-53)
mark_inset(ax1, axins, loc1=1, loc2=3, fc="none", ec="0.25")

axins.annotate('3', xy=(7.48,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
axins.annotate('4', xy=((7.5+7.53)/2,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
axins.annotate('5', xy=(7.55,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))

ax1.set_xlim(-0.15, 10.75)
ax1.set_ylim(0.00005, 1000)
ax2.set_ylim(-0, 135)

plt.tight_layout()
subplots_adjust(hspace=0.0,bottom=0.11,left=0.12,right=0.98,top=0.97)
# savefig(fname = figname+'_'+re.sub('/','_',dirname_hc[31:-1])+'.png',dpi=300)
# savefig(figname+'_D'+str(Detuning)+'_50eV'+'.png',dpi=300)

/home/adrian/.local/lib/python3.6/site-packages/matplotlib/figure.py:2267: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


In [469]:
'20200903/DC05_RF10/Try28'
# tt*1e3,savgol_filter(mean(T_all,axis=1),25,1
# ax2.scatter(tt_1ms_step,PM_1ms, label='cts/ms',marker='P',s=200,lw=1.5,color='black',zorder=100)
# ax2.plot(tt_1us_step[:-500]*1e3,PM_1us[:-500], label='cts/µs',color='blue')
outfile = 'graph_example_20200903_DC05_RF10_Try28'
# savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)
np.savez(outfile, time=tt, temperature=T_all, time_1us=tt_1us_step, time_1ms=tt_1ms_step, PM_1us=PM_1us, PM_1ms=PM_1ms)

In [471]:
# load check
outfile = 'graph_example_20200903_DC05_RF10_Try28'
with load(outfile+'.npz') as data:
    a = data['PM_1ms']
print(a)

[72.84216 72.75278 72.66428 72.38821 72.24065 72.16199 27.70817 14.02974]


In [482]:
savefig(figname+'.eps',dpi=600,format='eps')
savefig(figname+'_D'+str(Detuning)+'_50eV'+'.png',dpi=300)

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

# Plot all temp for all conditions
fname = 'Temp_allpoints'
cm = pylab.get_cmap('rainbow')

fig, ax = plt.subplots(4, 7,num=fname,clear=True,sharex=True,sharey=True)  # (a,b)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')

to_plot = 0

for l in range(0,7): # b
    for k in range(0,4): # a
        plot_point = to_plot
#             str_eta_chiffre    = float(f'{eta[patate]:1.2e}'[:4])/10
#             str_eta_puissance  = int(re.sub('e','D',f'{eta[patate]:1.2e}'[6:]))-1
#             str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
        str_eta = '0.15D-19'
        
        for plot_try in range(25):
            dirname_hc = str(data0[plot_point][plot_try])+'/'
            # Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
            datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
            N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T = \
            load_Temp_init_bin_Lan(dirname_hc+datainit_hc,flag_print=0)
            # Get T and fluo PM
            onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and 'SimuType' in f and '.dat' in f]
            ta, te, ti = plot_T_and_PM_InitQ_Inje_Evol(dirname_hc,sort(onlyfiles)[2].strip('.dat')[4:],flag_plot = 0,fig_name = 'Temp_allpoints')

            tt_Lan = linspace(0,dt_j_save_next*j_save,j_save)
            tt_md = ta[0]
            T_aux = te[0]

    #         tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
    #         tt_md = ta[0]
    #         T_aux = te[0]
    #         tt = concatenate((tt_Lan,tt_md))
    #         T_all = concatenate((save_T[:,4:],T_aux),axis=0)

            tt = concatenate((tt_Lan,tt_md))
            T_all = concatenate((save_T[:,4:],T_aux),axis=0)

#             ax[3-k,l].semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
#             ax[3-k,l].semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
            ax[3-k,l].semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
#         ax[k,l].vlines(ta[1]*1e3,1e-4,1e4,color='red')
#         ax[k,l].vlines(ta[2]*1e3,1e-4,1e4,color='red')
# print(i_free__fly_Lan)
# print(i_laser_fly_quench)
# print(i_laser_fly)
        time_switch = i_free__fly_Lan*dt
        ax[3-k,l].vlines(time_switch*1e3,1e-4,1e4,color='blue')
        time_switch = (i_free__fly_Lan+i_laser_fly)*dt
        ax[3-k,l].vlines(time_switch*1e3,1e-4,1e4,color='green')
        time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
        ax[3-k,l].vlines(time_switch*1e3,1e-4,1e4,color='orange')
        time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
        ax[3-k,l].vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
        ax[3-k,l].set_yticks([1e-4,1e-2,1,1e2,1e4])
        ax[3-k,l].grid()
        ax[3-k,l].set_xlim(7.49,7.57)
        im = ax[3-k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.04))
        ax[3-k,l].annotate(str(list(dict.fromkeys(points_and_coord))[plot_point]), xy=(0.20, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
        size=12, ha='left', va='top',
        bbox=dict(boxstyle='round', fc=cm(detect_rate[to_plot])))
        to_plot += 1
        if to_plot == 28:
            break
    if to_plot == 28:
        break
# ax[3].set_xlim(-10,4000)
# ax[k,l].set_ylim(2e-4,1e4)
fig.suptitle(str(eta) + 'Kg.s-1')
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

# fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig(fname+'.png') 

In [ ]:
# Plot all fluo for all conditions
fname = 'Fluo_allpoints'
cm = pylab.get_cmap('rainbow')

fig, ax = plt.subplots(4, 7,num=fname,clear=True,sharex=True,sharey=True)  # (a,b)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')

to_plot = 0

for l in range(0,7): # b
    for k in range(0,4): # a
        plot_point = to_plot
#             str_eta_chiffre    = float(f'{eta[patate]:1.2e}'[:4])/10
#             str_eta_puissance  = int(re.sub('e','D',f'{eta[patate]:1.2e}'[6:]))-1
#             str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
        str_eta = '0.15D-19'
    
        for plot_try in range(25):
            dirname_hc = str(data0[plot_point][plot_try])+'/'
            # Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
            datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
            N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T = \
            load_Temp_init_bin_Lan(dirname_hc+datainit_hc,flag_print=0)
            # Get T and fluo PM
            onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and 'SimuType' in f and '.dat' in f]
            ta, te, ti = plot_T_and_PM_InitQ_Inje_Evol(dirname_hc,sort(onlyfiles)[2].strip('.dat')[4:],flag_plot = 0,fig_name = 'Temp_allpoints')

            tt_Lan = linspace(0,dt_j_save_next*j_save,j_save)
            tt_md = ta[0]
            T_aux = ti[0]
        
#         tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
#         tt_md = ta[0]
#         T_aux = te[0]
#         tt = concatenate((tt_Lan,tt_md))
#         T_all = concatenate((save_T[:,4:],T_aux),axis=0)

            tt = tt_md
            T_all = T_aux

            ax[3-k,l].plot(tt*1e3,T_aux, label='Fluo',color='C0')
#         ax[3-k,l].semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
#         ax[3-k,l].semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
#         ax[k,l].vlines(ta[1]*1e3,1e-4,1e4,color='red')
#         ax[k,l].vlines(ta[2]*1e3,1e-4,1e4,color='red')
# print(i_free__fly_Lan)
# print(i_laser_fly_quench)
# print(i_laser_fly)
        time_switch = i_free__fly_Lan*dt
        ax[3-k,l].vlines(time_switch*1e3,1e-4,1e4,color='blue')
        time_switch = (i_free__fly_Lan+i_laser_fly)*dt
        ax[3-k,l].vlines(time_switch*1e3,1e-4,1e4,color='green')
        time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
        ax[3-k,l].vlines(time_switch*1e3,1e-4,1e4,color='orange')
        time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
        ax[3-k,l].vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
        ax[3-k,l].grid()
        im = ax[3-k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.2))
        ax[3-k,l].annotate(str(list(dict.fromkeys(points_and_coord))[plot_point]), xy=(0.20, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
        size=12, ha='left', va='top',
        bbox=dict(boxstyle='round', fc=cm(detect_rate[to_plot])))
        to_plot += 1
        if to_plot == 28:
            break
    if to_plot == 28:
        break
# ax[3].set_xlim(-10,4000)
ax[3-k,l].set_xlim(7.24,7.80)
ax[3-k,l].set_ylim(0,100)
fig.suptitle(str(eta) + 'Kg.s-1')
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

# fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig(fname+'.png')

In [53]:
# delta Ec
renorm = 0.5*m_GM/C_e
k = 10
l = 0
print(data0[k][l])
print(Gmol_data[0][k][l]*renorm*1e3, 'meV')

/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try00
-22.553567767578976 meV


In [19]:
Gmol_data[2]

mean_std_deltaEc = []
detect_rate = []
detect_cnt = 0
detect_threshold = 1
renorm = 0.5*m_GM/C_e

# mean_std_deltaEc = [mean(Gmol_data[0])*renorm,std(Gmol_data[0])*renorm,mean(detec_delta_Ec)*renorm,std(detec_delta_Ec)*renorm]

for k in range(shape(Gmol_data[2])[0]):
    detec_delta_Ec = []
    print('k',k)
    detect_rate.append(0)
    for l,m in enumerate(Gmol_data[2][k]):
        print('... l',l)
        if m > detect_threshold:
            detect_cnt += 1
            detect_rate[k] += 1
            detec_delta_Ec.append(Gmol_data[0][k][l])
    detect_rate[k] = detect_rate[k] / shape(Gmol_data[2])[1]
    mean_std_deltaEc.append(list(multiply([mean(Gmol_data[0][k]),std(Gmol_data[0][k]),mean(detec_delta_Ec),std(detec_delta_Ec)],renorm)))
    
for n,o in enumerate(list(dict.fromkeys(points_and_coord))):
    print(o)
    print(detect_rate[n])

k 0
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
... l 6
... l 7
... l 8
... l 9
... l 10
... l 11
... l 12
... l 13
... l 14
... l 15
... l 16
... l 17
... l 18
... l 19
... l 20
... l 21
... l 22
... l 23
... l 24
... l 25
... l 26
... l 27
... l 28
... l 29
... l 30
... l 31
... l 32
... l 33
... l 34
... l 35
... l 36
... l 37
... l 38
... l 39
... l 40
... l 41
... l 42
... l 43
... l 44
... l 45
... l 46
... l 47
... l 48
... l 49
... l 50
... l 51
... l 52
... l 53
... l 54
... l 55
... l 56
... l 57
... l 58
... l 59
... l 60
... l 61
... l 62
... l 63
... l 64
... l 65
... l 66
... l 67
... l 68
... l 69
... l 70
... l 71
... l 72
... l 73
... l 74
... l 75
... l 76
... l 77
... l 78
... l 79
... l 80
... l 81
... l 82
... l 83
... l 84
... l 85
... l 86
... l 87
... l 88
... l 89
... l 90
... l 91
... l 92
... l 93
... l 94
... l 95
... l 96
... l 97
... l 98
... l 99
k 1
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
... l 6
... l 7
... l 8
... l 9
... l 10
... l 11
... 

/home/adrian/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/adrian/.local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/adrian/.local/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/adrian/.local/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/adrian/.local/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



... l 71
... l 72
... l 73
... l 74
... l 75
... l 76
... l 77
... l 78
... l 79
... l 80
... l 81
... l 82
... l 83
... l 84
... l 85
... l 86
... l 87
... l 88
... l 89
... l 90
... l 91
... l 92
... l 93
... l 94
... l 95
... l 96
... l 97
... l 98
... l 99
k 11
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
... l 6
... l 7
... l 8
... l 9
... l 10
... l 11
... l 12
... l 13
... l 14
... l 15
... l 16
... l 17
... l 18
... l 19
... l 20
... l 21
... l 22
... l 23
... l 24
... l 25
... l 26
... l 27
... l 28
... l 29
... l 30
... l 31
... l 32
... l 33
... l 34
... l 35
... l 36
... l 37
... l 38
... l 39
... l 40
... l 41
... l 42
... l 43
... l 44
... l 45
... l 46
... l 47
... l 48
... l 49
... l 50
... l 51
... l 52
... l 53
... l 54
... l 55
... l 56
... l 57
... l 58
... l 59
... l 60
... l 61
... l 62
... l 63
... l 64
... l 65
... l 66
... l 67
... l 68
... l 69
... l 70
... l 71
... l 72
... l 73
... l 74
... l 75
... l 76
... l 77
... l 78
... l 79
... l 80
... l 81
... l

... l 71
... l 72
... l 73
... l 74
... l 75
... l 76
... l 77
... l 78
... l 79
... l 80
... l 81
... l 82
... l 83
... l 84
... l 85
... l 86
... l 87
... l 88
... l 89
... l 90
... l 91
... l 92
... l 93
... l 94
... l 95
... l 96
... l 97
... l 98
... l 99
k 24
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
... l 6
... l 7
... l 8
... l 9
... l 10
... l 11
... l 12
... l 13
... l 14
... l 15
... l 16
... l 17
... l 18
... l 19
... l 20
... l 21
... l 22
... l 23
... l 24
... l 25
... l 26
... l 27
... l 28
... l 29
... l 30
... l 31
... l 32
... l 33
... l 34
... l 35
... l 36
... l 37
... l 38
... l 39
... l 40
... l 41
... l 42
... l 43
... l 44
... l 45
... l 46
... l 47
... l 48
... l 49
... l 50
... l 51
... l 52
... l 53
... l 54
... l 55
... l 56
... l 57
... l 58
... l 59
... l 60
... l 61
... l 62
... l 63
... l 64
... l 65
... l 66
... l 67
... l 68
... l 69
... l 70
... l 71
... l 72
... l 73
... l 74
... l 75
... l 76
... l 77
... l 78
... l 79
... l 80
... l 81
... l 

In [54]:
# delta Ec moy pour toutes les simus
print(mean(Gmol_data[0])*renorm)
print(std(Gmol_data[0])*renorm)

# delta Ec moy quand SNR > 1
detec_delta_Ec = []
for i in range(shape(Gmol_data[2])[0]):
    for j in range(shape(Gmol_data[2])[1]):
        if Gmol_data[2][i][j] > 1:
            detec_delta_Ec.append(Gmol_data[0][i][j])
# print(detec_delta_Ec)
print(len(detec_delta_Ec))
print(mean(detec_delta_Ec)*renorm)
print(std(detec_delta_Ec)*renorm)

-0.017224867763344646
0.012052917433834532
1562
-0.023386368006814807
0.012080536632384611


In [55]:
# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

1.0


In [56]:
Udc = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 7.5, 3.33, 3.66, 4.5, 5.33, 5.66, 6, 6.5]
Urf = [10.77, 16.15, 21.54, 26.92, 32.31, 37.69, 43.08, 48.46, 53.85, 59.23, 64.61,
       70.00, 75.38, 80.77, 61.02, 62.82, 66.41, 68.20, 71.79, 55.64, 57.43]

In [64]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

In [58]:
udc_var = linspace(0.001,8.2,1e3)
urf_low_lim  = sqrt( udc_var * kappa_simion*Omega**2*m_Ca*r0**4/ (2*C_e*d0**2) )
urf_high_lim = m_Ca*r0**2*Omega**2/4/C_e*0.9

my_omegaz_2 = 2*kappa_simion*C_e*udc_var/m_Ca/d0**2
my_omegap_2 = 0.5 * 2*kappa_simion*C_e*udc_var/m_Ca/d0**2
my_omegar_2 = my_omegap_2 - 0.5


max_q = 0.908
max_omegap_2 = max_q**2 * Omega**2 / 8
max_omegar_2 = max_omegap_2 - 0.5*my_omegaz_2

/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  """Entry point for launching an IPython kernel.


In [601]:
detect_rate[3:28:4]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.92]

In [626]:
for kk in range(4):
    print(z[cluster==4-kk])

[100 100 100 100 100 100  92]
[100 100  98  71  34  39  86]
[100  99  30   5  31   0   0]
[63 14  0  0  0  0  0]


In [629]:
x,y,z

(array([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 7, 7,
        7, 7, 6, 6, 6, 6]),
 array([53.85, 59.23, 64.61, 70.  , 53.85, 59.23, 64.61, 70.  , 53.85,
        59.23, 64.61, 70.  , 53.85, 59.23, 64.61, 70.  , 53.85, 59.23,
        64.61, 70.  , 53.85, 59.23, 64.61, 70.  , 53.85, 59.23, 64.61,
        70.  ]),
 array([ 63, 100, 100, 100,  14,  99, 100, 100,   0,  30,  98, 100,   0,
          5,  71, 100,   0,  31,  34, 100,   0,   0,  39, 100,   0,   0,
         86,  92]))

In [650]:
list(x[cluster==kk+1])+[0,0]

[1, 2, 3, 4, 5, 7, 6, 0, 0]

In [656]:
x = []
y = []
z = []

for g,h in enumerate(points_and_coord):
    my_udc = Udc[int(points_and_coord[h][0])]
    my_urf = Urf[int(points_and_coord[h][1])]
    x.append(my_udc)
    y.append(my_urf)
    z.append(detect_rate[g])
    
cluster = array([1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4])
my_marker = ['v','s','p','H']

x=np.array(x)
y=np.array(y)
z=np.array(multiply(z,100),dtype='int')

fname = '2D_GMol_var_fluo'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
# title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

for kk in range(4):
    im = ax.scatter(list(x[cluster==kk+1])+[0,0],list(y[cluster==kk+1])+[0,0],
        marker=my_marker[kk%4],s=600,edgecolor='black',zorder = 1000,
        c = list(z[cluster==kk+1])+[0,100], cmap='viridis') # discrete_cmap(len(num_runs), 'jet')
ax.set_xlabel(r'$U_{DC}$ [V]')
ax.set_ylabel(r'$U_{RF}$ [V]')
ax.grid()

imbar = ax.scatter(x,y,
        marker=my_marker[1],s=0,edgecolor='black',zorder = 1000,
        c = z, cmap='viridis')
cbar = plt.colorbar(imbar, ax=ax)
# cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
cbar.ax.set_ylabel(r'Detection rate [%]', rotation=90)

ax.set_xlim(0.12,7.2)
ax.set_ylim(52.5,71.5)
ax.annotate('$q_x=0.6$', (0.22,64.23),fontsize=18)
ax.annotate('$q_x=0.5$', (0.22,53.55),fontsize=18)

# ax.grid()
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
fig.set_size_inches(14,6)

fig.tight_layout()
subplots_adjust(hspace=0.10, bottom=0.163, top = 0.93,left=0.08,right=1.04)
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=600)
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.eps',dpi=600,format='eps')

In [212]:
outfile = '2D_GMol_var_fluo'+str(E0_GMol)+'eV'
# savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)
np.savez(outfile, x=my_udc, y=my_urf, z=detect_rate)
#     my_udc = Udc[int(points_and_coord[h][0])]
#     my_urf = Urf[int(points_and_coord[h][1])]
#     x.append(my_udc)
#     y.append(my_urf)
#     z.append(detect_rate[g])

In [653]:
x = []
y = []
z = []

for g,h in enumerate(points_and_coord):
    my_udc = Udc[int(points_and_coord[h][0])]
    my_urf = Urf[int(points_and_coord[h][1])]
    x.append(my_udc)
    y.append(my_urf)
    z.append(detect_rate[g])

    
fname = '2D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

im = ax.scatter(x,y,
        marker='D',s=400,edgecolor='black',zorder = 1000,
        c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))
ax.plot(udc_var,urf_low_lim)
ax.plot(udc_var,[urf_high_lim]*len(udc_var))
ax.set_xlabel(r'$U_{DC}$ [V]')
ax.set_ylabel(r'$U_{RF}$ [V]')
ax.grid()

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=13,
                ha='center',va='center',weight='bold',zorder = 1000)
    ax.annotate(str(txt), (x[i], y[i]+4e4),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)

In [ ]:
r0,d0
walaybilai = my_omegaz_2*d0**2/r0**2*0.25/100000

In [ ]:
x = []
y = []
z = []
omegaz_2 = zeros(len(list(points_and_coord)))
omegar_2 = zeros(len(list(points_and_coord)))
omegap_2 = zeros(len(list(points_and_coord)))
for g,h in enumerate(points_and_coord):
    my_udc = Udc[int(points_and_coord[h][0])]
    my_urf = Urf[int(points_and_coord[h][1])]
#     x.append(my_udc)
#     y.append(my_urf)
    z.append(detect_rate[g])
    
    omegaz_2[g] = 2*kappa_simion*C_e/m_Ca/d0**2*my_udc
    omegap_2[g] = 2*C_e**2/m_Ca**2/r0**4/Omega**2*my_urf**2
    omegar_2[g] = omegap_2[g] - 0.5*omegaz_2[g]
    x.append(sqrt(omegaz_2[g]))
    y.append(sqrt(omegar_2[g]))
    
fname = '2D_GMol_var_fluo_contour_wzwr'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

im = ax.scatter(x,y,
        marker='D',s=350,edgecolor='black',zorder = 1000,
        c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))
ax.plot(sqrt(my_omegaz_2),walaybilai)
# ax.plot(sqrt(my_omegaz_2),sqrt(max_omegar_2))
ax.set_xlabel(r'$\omega_{z}$ [Hz]')
ax.set_ylabel(r'$\omega_{r}$ [Hz]')
ax.grid()

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=12,
                ha='center',va='center',weight='bold',zorder = 1000)
#     ax.annotate(str(txt), (x[i], y[i]+4e4),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5e6))
fig.set_size_inches(11.69, 8.27)

# ax.set_ylim(-50000,4250000)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)

In [ ]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(mean(dim_nu[g,:,0])/mean(dim_nu[g,:,2]))
    y.append(mean(dim_nu[g,:,0])*1000)
    z.append(detect_rate[g])
    
    
fname = '2D_GMol_var_fluo_contour_Rvs_RoverL'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

im = ax.scatter(x,y,
        marker='D',s=250,edgecolor='black',zorder = 1000,
        c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))
ax.set_xlabel(r'mean R/L')
ax.set_ylabel(r'mean R [µm]')

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=10,
                ha='center',va='center',weight='bold',zorder = 1000)
    # ax.annotate(str(txt), (x[i], y[i]-0.70),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.05))
ax.minorticks_on()
# ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.grid(which='major', linestyle='-', linewidth='1', color='k')
ax.grid(b=True, which='minor', color='k', linestyle=':')
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)

In [574]:
shape(dim_nu)
print(dim_nu[10,28])
rayon = (dim_nu[10,28,0]+dim_nu[10,28,1])/2
print(rayon)

[0.0467127  0.04683864 0.23460626]
0.04677566713029322


In [566]:
address = all_subdir[100*10+28]
print(address)
# dirname_hc = str(data0[1][0])+'/'
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and 'SimuType' in f and '.dat' in f]
my_file = '{}/xva{}'.format(address,sort(onlyfiles)[2].strip('.dat')[4:])
plot_XYZ(my_file,fig_name='2',fig_title='XYZ')

/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try28


In [ ]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(mean(dim_nu[g,:,0])/mean(dim_nu[g,:,2]))
    y.append(mean(dim_nu[g,:,2])*1000)
    z.append(detect_rate[g])
    
    
fname = '2D_GMol_var_fluo_contour_Lvs_RoverL'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

im = ax.scatter(x,y,
        marker='D',s=250,edgecolor='black',zorder = 1000,
        c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))
ax.set_xlabel(r'mean R/L')
ax.set_ylabel(r'mean L [µm]')

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=10,
                ha='center',va='center',weight='bold',zorder = 1000)
    # ax.annotate(str(txt), (x[i], y[i]-0.70),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.05))
ax.minorticks_on()
# ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.grid(which='major', linestyle='-', linewidth='1', color='k')
ax.grid(b=True, which='minor', color='k', linestyle=':')
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)

In [ ]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(mean(dim_nu[g,:,0])*1000)
    y.append(mean(dim_nu[g,:,2])*1000)
    z.append(detect_rate[g])
    
    
fname = '2D_GMol_var_fluo_contour_RvsL'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

im = ax.scatter(x,y,
        marker='D',s=250,edgecolor='black',zorder = 1000,
        c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))
ax.set_xlabel(r'mean R [µm]')
ax.set_ylabel(r'mean L [µm]')

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=10,
                ha='center',va='center',weight='bold',zorder = 1000)
    # ax.annotate(str(txt), (x[i], y[i]-0.70),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.01*1000))
ax.minorticks_on()
# ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.grid(which='major', linestyle='-', linewidth='1', color='k')
ax.grid(b=True, which='minor', color='k', linestyle=':')
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)

# SNR détaillé

In [ ]:
plt.close('all')

In [ ]:
# Recover Ca pos, speed and acc at the end of one specific step
# address = simutype6address[20]

# my_plot = 1
# for my_plot,ll in enumerate(list(dict.fromkeys(points_and_coord))):

my_plot = 26

print(my_plot,ll)
my_base = 0.1

fname = 'SNR_'+'#'+str(my_plot)+'_'+ll
fig = figure(fname); clf()
fig.suptitle(r'SNR '+ll,color='xkcd:black',fontsize = 18)
ax1 = subplot(111)
my_bins = arange(4.65,5.25,0.005)
ax1.hist(Gmol_data[2][my_plot],bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax1.grid()
xlabel('SNR')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)
savefig(fname+'.png') 

In [ ]:
# Plot all fluo for all conditions
fname = 'DeltaEc_allpoints'
cm = pylab.get_cmap('rainbow')

fig, ax = plt.subplots(4, 7,num=fname,clear=True,sharex=True,sharey=True)  # (a,b)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')

my_order = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,24,25,26,27,20,21,22,23]

my_base = 40
to_plot = 0
for l in range(0,7): # b
    for k in range(0,4): # a
        plot_point = my_order[to_plot]
#             str_eta_chiffre    = float(f'{eta[patate]:1.2e}'[:4])/10
#             str_eta_puissance  = int(re.sub('e','D',f'{eta[patate]:1.2e}'[6:]))-1
#             str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
        str_eta = '0.15D-19'

        my_bins = arange(-71,10,5)
        ax[3-k,l].hist(Gmol_data[0][plot_point]*renorm*1e3,bins=my_bins ,color='C5', edgecolor='black', linewidth=1.2)            
        ax[3-k,l].annotate(str(list(dict.fromkeys(points_and_coord))[plot_point]), xy=(0.02, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
        size=12, ha='left', va='top',
        bbox=dict(boxstyle='round', fc=cm(detect_rate[to_plot])))
        ax[3-k,l].grid()
        to_plot += 1
        if to_plot == 28:
            break
    if to_plot == 28:
        break
ax[3-k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax[3-k,l].yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=20))
# ax[3].set_xlim(-10,4000)
# ax[3-k,l].set_xlim(7.24,7.80)
ax[3-k,l].set_ylim(0,81)
# fig.suptitle(str(eta) + 'Kg.s-1')
fig.text(0.51, 0.04, r'$\Delta E_c$ [meV]', ha='center')
# fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

# fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.05, top = 0.98) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)

# GMol trajectory

In [ ]:
os.path.join(address_to_plot[0],trajectory_files[0][0]).strip('.bin')

In [ ]:
# Choose a specific point to consider
# Choose one point on the plot
plot_point = 4
condition_to_respect = list(dict.fromkeys(points_and_coord))[plot_point]
address_to_plot = [l for k,l in enumerate(all_subdir) if condition_to_respect in l]
for j,k in enumerate(address_to_plot):
    temp_files = listdir(k)
# Recover all GMol trajectories addresses
trajectory_files  = [[m for l,m in enumerate(temp_files) if 'trj' in m] for j,k in enumerate(address_to_plot)]
temperature_files = [[m for l,m in enumerate(temp_files) if 'Temp_SimuType4' in m] for j,k in enumerate(address_to_plot)]
    
#trajectory_file = [listdir(j) for i,j in enumerate(address_to_plot)]
#trajectory_file = [k for j,k in enumerate(trajectory_file) if 'trj' in k]
# print(trajectory_file)

temp_size = len(load_trj(os.path.join(address_to_plot[0],trajectory_files[0][0]).strip('.bin'))[0])
t, r_x,r_y,r_z, v_x,v_y,v_z, a_x,a_y,a_z = np.zeros((10,len(trajectory_files),temp_size))
tt, PM      = np.zeros((2,len(temperature_files),temp_size))
T_CM, T_aux = np.zeros((2,len(temperature_files),temp_size,3))
for j,k in enumerate(trajectory_files): # try
    print('>',j,'-',k)
    t[j], r_x[j],r_y[j],r_z[j], v_x[j],v_y[j],v_z[j], a_x[j],a_y[j],a_z[j] = load_trj(os.path.join(address_to_plot[j],k[0].strip('.bin')))
for j,k in enumerate(temperature_files): # try
    print('>',j,'-',k)
    tt[j], T_CM[j], T_aux[j], PM[j] = load_T_and_PM_simu(os.path.join(address_to_plot[j],k[0].strip('.dat')))

SNR_trig_threshold = 1

fname = 'plot_one_trj_'+condition_to_respect+'_'+str(E0_GMol)+'eV'
fig, ax = plt.subplots(2,1,num=fname,clear=True)

for k in range(len(r_z)):
    ax[0].plot(r_z[k]*1000,r_x[k]*1e9)
    ax[1].plot(r_z[k]*1000,r_y[k]*1e9)
    ax[1].set_xlabel('z [mm]')
    ax[0].set_ylabel('x [nm]')
    ax[1].set_ylabel('y [nm]')
    ax[0].set_ylim(-520,520)
    ax[1].set_ylim(-520,520)
ax[1].grid()
ax[0].grid()
fig.set_size_inches(11.69, 8.27)
tight_layout()

fname = 'plot_one_spexy_'+condition_to_respect+'_'+str(E0_GMol)+'eV'
fig, ax = plt.subplots(2,1,num=fname,clear=True)

for k in range(len(r_z)):
    ax[0].plot(r_z[k]*1000,v_x[k])
    ax[1].plot(r_z[k]*1000,v_y[k])
    ax[1].set_xlabel('z [mm]')
    ax[0].set_ylabel('v_x [m/s]')
    ax[1].set_ylabel('v_y [m/s]')
#     ax[0].set_ylim(-520,520)
#     ax[1].set_ylim(-520,520)
    ax[1].grid()
    ax[0].grid()
fig.set_size_inches(11.69, 8.27)
tight_layout()

fname = 'plot_one_spez_'+condition_to_respect+'_'+str(E0_GMol)+'eV'
fig, ax = plt.subplots(1,1,num=fname,clear=True)

for k in range(len(r_z)):
    ax.plot(r_z[k]*1000,v_z[k])
    ax.set_xlabel('z [mm]')
    ax.set_ylabel('v_z [m/s]')
#     ax[0].set_ylim(-520,520)
#     ax[1].set_ylim(-520,520)
    ax.grid()
fig.set_size_inches(11.69, 8.27)
tight_layout()

# Distribution des vitesses

In [ ]:
address = all_subdir[22]
# dirname_hc = str(data0[1][0])+'/'
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and 'SimuType' in f and '.dat' in f]
my_file = '{}/xva{}'.format(address,sort(onlyfiles)[2].strip('.dat')[4:])
print(my_file)
r_LC,v_LC,a_LC,v_LC_avg = load_xyz_init_bin_DP(my_file)  

In [ ]:
# Recover Ca pos, speed and acc at the end of one specific step
# address = simutype6address[20]
print(address)
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and "xva" in f and 'SimuTypeQ' in f and 'bin' in f]
r_LC, v_LC, a_LC, v_rf_avg = load_xyz_init_bin_DP(address+'/'+onlyfiles[0].strip('.bin'))


v_LC_x = v_LC[0]
v_LC_y = v_LC[1]
v_LC_z = v_LC[2]

my_base = 50

fig = figure('Terminal speed'); clf()
fig.suptitle(r'Atomic speed at the end of SimuType0 - RF - '+str(address[31:]),color='xkcd:black',fontsize = 18)
ax1 = subplot(311)
my_bins = arange(-250,251,20)
ax1.hist(v_LC_x,bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax1.grid()
xlabel('$v_x$ [m/s]')

ax2 = subplot(312)
ax2.hist(v_LC_y,bins=my_bins ,color='C1', edgecolor='black', linewidth=1.2)
ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax2.grid()
xlabel('$v_y$ [m/s]')

my_base = 0.2
ax3 = subplot(313)
my_bins = arange(-2,2.1,0.1)
# my_bins = arange(-30000,30001,5000)
ax3.hist(v_LC_z,bins=my_bins ,color='C2', edgecolor='black', linewidth=1.2)
ax3.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax3.grid()
xlabel('$v_z$ [m/s]')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)

In [ ]:
# Recover Ca pos, speed and acc at the end of one specific step
# address = simutype6address[20]PleasePl
print(address)
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and "xva" in f and 'SimuTypeQ' in f and 'bin' in f]
r_LC, v_LC, a_LC, v_rf_avg = load_xyz_init_bin_DP(address+'/'+onlyfiles[0].strip('.bin'))
print(address+'/'+onlyfiles[0])


v_LC_avg_x = v_rf_avg[0]/100
v_LC_avg_y = v_rf_avg[1]/100
v_LC_avg_z = v_rf_avg[2]/100

my_base = 5

fig = figure('Terminal speed 2'); clf()
fig.suptitle(r'Atomic speed at the end of SimuType0 - no RF - '+str(address[31:]),color='xkcd:black',fontsize = 18)
ax1 = subplot(311)
my_bins = arange(-25,26,2)
ax1.hist(v_LC_avg_x,bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax1.grid()
xlabel('$v_x$ [m/s]')

ax2 = subplot(312)
ax2.hist(v_LC_avg_y,bins=my_bins ,color='C1', edgecolor='black', linewidth=1.2)
ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax2.grid()
xlabel('$v_y$ [m/s]')

my_base = 0.2
ax3 = subplot(313)
my_bins = arange(-2,2.1,0.1)
# my_bins = arange(-30000,30001,5000)
ax3.hist(v_LC_avg_z,bins=my_bins ,color='C2', edgecolor='black', linewidth=1.2)
ax3.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax3.grid()
xlabel('$v_z$ [m/s]')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)

In [ ]:
m_kb_x_inv_n_ions  = m_Ca/(kb*N_ions*n_dt**2   )
T_aux_LC_x = sum(v_rf_avg[0]**2)*m_kb_x_inv_n_ions
T_aux_LC_y = sum(v_rf_avg[1]**2)*m_kb_x_inv_n_ions
T_aux_LC_z = sum(v_rf_avg[2]**2)*m_kb_x_inv_n_ions

fig = figure('Calculated temp'); clf()
fig.suptitle(r'Atomic speed at the end of SimuType0 - no RF - '+str(address[31:]),color='xkcd:black',fontsize = 18)

my_base = 1
my_bins = arange(0,1,0.005)

ax1 = subplot(311)
ax1.hist(T_aux_LC_x,bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax1.grid()
xlabel('$v_x$ [m/s]')

ax2 = subplot(312)
ax2.hist(T_aux_LC_y,bins=my_bins ,color='C1', edgecolor='black', linewidth=1.2)
ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax2.grid()
xlabel('$v_y$ [m/s]')

ax3 = subplot(313)
# my_bins = arange(-2,2.1,0.1)
# my_bins = arange(-30000,30001,5000)
ax3.hist(T_aux_LC_z,bins=my_bins ,color='C2', edgecolor='black', linewidth=1.2)
ax3.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax3.grid()
xlabel('$v_z$ [m/s]')

fig.set_size_inches(11.69, 8.27)
# plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)

In [ ]:
n_threads = 32
space = N_ions/n_threads
linspace(1,1024-space+1,n_threads,dtype='int'),linspace(space,1024,n_threads,dtype='int')

# Ion position

In [483]:
my_point = 10
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and 'SimuType' in f and '.dat' in f]
my_file = '{}/xva{}'.format(address,sort(onlyfiles)[2].strip('.dat')[4:])
r_LC,_,_,_ = load_xyz_init_bin_DP(my_file)
r_LC = np.zeros((len(num_runs),3,len(r_LC[0])))
for k,address in enumerate(all_subdir[my_point*len(num_runs):(my_point+1)*len(num_runs)]): # my_point*len(num_runs),(my_point+1)*len(num_runs)):
#     print(address)
    onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and 'SimuType' in f and '.dat' in f]
    my_file = '{}/xva{}'.format(address,sort(onlyfiles)[2].strip('.dat')[4:])
    print(my_file)
    r_LC[k],_,_,_ = load_xyz_init_bin_DP(my_file)

/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try00/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try01/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try02/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try03/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try04/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try05/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try06/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try07/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try08/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+

/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try88/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try89/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try90/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try91/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try92/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try93/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try94/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try95/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC05_RF10/Try96/xva_SimuTypeQ_N01024_Vrf0064_Udc0.3000D+

In [487]:
# Recover Ca pos, speed and acc at the end of one specific step
# address = simutype6address[20]PleasePl
my_base = 28

fig = figure('Ion pos at end cooling'); clf()
fig.suptitle(r'Ion pos at end cooling - '+str(address[31:]),color='xkcd:black',fontsize = 18)


ion = linspace(0,1023,1024,dtype='int')

for k in range(1):
    ax1 = subplot(311)
    my_bins = arange(-25,26,2)
    ax1.scatter(r_LC[k,2,ion]*1e3,r_LC[k,0,ion]*1e3, edgecolor='black', linewidth=1.2) # color='C0',
    ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
    ax1.axis('equal')
    ax1.grid()
    xlabel('$z$ [µm]')
    ylabel('$x$ [µm]')

    ax2 = subplot(312)
    ax2.scatter(r_LC[k,2,ion]*1e3,r_LC[k,1,ion]*1e3, edgecolor='black', linewidth=1.2)
    ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
    ax2.axis('equal')
    ax2.grid()
    xlabel('$z$ [µm]')
    ylabel('$y$ [µm]')

    ax3 = subplot(313)
    ax3.scatter(r_LC[k,0,ion]*1e3,r_LC[k,1,ion]*1e3, edgecolor='black', linewidth=1.2)
    ax3.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
    ax3.set_aspect('equal', 'box')
    ax3.grid()
    xlabel('$x$ [µm]')
    ylabel('$y$ [µm]')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)

In [ ]:
ion = linspace(0,1023,1024,dtype='int')
k = 0

fig = figure('Ion pos at end cooling - slicing - 3D'); clf()
ax1 = fig.add_subplot(111, projection='3d')

lower_x = -0.3
upper_x = +0.3

for k in range(25):
    x_slice = [x_pos*1e3 for n_atom,x_pos in enumerate(r_LC[k,0,:]) if lower_x<x_pos*1000<upper_x]
    y_slice = [y_pos*1e3 for n_atom,y_pos in enumerate(r_LC[k,1,:]) if lower_x<r_LC[k,0,n_atom]*1e3<upper_x]
    z_slice = [z_pos*1e3 for n_atom,z_pos in enumerate(r_LC[k,2,:]) if lower_x<r_LC[k,0,n_atom]*1e3<upper_x]
    ax1.scatter(x_slice,y_slice,z_slice, s=120, edgecolor='black', linewidth=2)
#     ax1.scatter(r_LC[k,0,ion]*1e3,r_LC[k,1,ion]*1e3,r_LC[k,2,ion]*1e3, edgecolor='black', linewidth=1.2)
#     ax1.axis('equal')
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_zlabel('z')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()

In [ ]:
ion = linspace(0,1023,1024,dtype='int')
k = 0

fig = figure('Ion pos at end cooling - 3D'); clf()
ax1 = fig.add_subplot(111, projection='3d')


for k in range(3):
    X = r_LC[k,0,ion]*1e3
    Y = r_LC[k,1,ion]*1e3
    Z = r_LC[k,2,ion]*1e3
    ax1.scatter(Z,Y,X, s=20, edgecolor='black', linewidth=1.2)
#     ax1.axis('equal')
#     ax1.set_xlabel('z')
#     ax1.set_ylabel('y')
#     ax1.set_zlabel('x')
    
ax1.set_xlim([-200,200])
ax1.set_ylim([-200,200])
ax1.set_zlim([-200,200])

plt.grid(b=None)
plt.axis('off')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()

In [ ]:
# depth axial potential
frac_E0_Ecr = 10
wz_LC = 2*pi*90806.9982303
rz_0 = -1.5e-3

wz2 = wz_LC**2 * m_Ca / m_GM

L     = 0.014  #(demi-longueur piège obtenue avec SIMION)
eta = 1. / (4.*(sqrt(10) + 5.)) # (~0.0306 je soupçonne cette valeur d'être le facteur d'écrantage)

Ep0   = wz2*eta*L**2*m_GM*(1- exp(rz_0**2 / (2*eta*L**2)) )
Ek_init = C_e * frac_E0_Ecr - Ep0

print(frac_E0_Ecr,Ep0/C_e,Ek_init/C_e)

In [ ]:
rz  = np.linspace(rz_0*10,-rz_0,1000)
my_udc
# Ep = wz2*eta*L**2*m_GM*(1- exp(rz_0**2 / (2*eta*L**2)) )
my_cte = -1/(2*eta*L**2)
Ep = wz2*my_udc*eta*L**2*m_GM*(1-exp(my_cte*rz**2))/C_e
Ep_real = 0.5*(wz2*m_GM)*rz**2/C_e*my_udc

fname = 'coucou'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
# title(all_subdir[0][:-15],fontsize=12)

ax.plot(rz*1000,Ep,label='Gaussien',lw=3)
ax.plot(rz[500:]*1000,Ep_real[500:],label='Harmonique',lw=3)
ax.vlines(rz_0*1000,0,1,label='Position initiale',lw=3,linestyle=':')
ax.set_xlabel('z [mm]')
ax.set_ylabel('$E_p$ [eV]')
ax.legend(title='Potentiel')

ax.grid()
tight_layout()

In [ ]:
Ep[0]-min(Ep)

# Detection rate vs. Incident energy

In [ ]:
i_Ec = [6, 10, 50, 100, 1000]
d_rate = multiply([213/392,283/479,1562/2800,411/840,97/588],100)

fig = figure('detection_rate_vs_incident_energy'); clf()
ax1 = fig.add_subplot(111)


for k in range(3):
    ax1.semilogx(i_Ec,d_rate, marker='D', ms=15, color='C1', markeredgecolor='k', linestyle='')
ax1.set_xlabel('Incident energy [eV]')
ax1.set_ylabel('Detection rate [%]')

matplotlib.pyplot.grid(True, which="both")
# ax1.xaxis.set_major_locator(plt.MaxNLocator(25))
fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
savefig('detection_rate_vs_incident_energy',dpi=300)

# ax.scatter(x,y,
#         marker='D',s=400,edgecolor='black',zorder = 1000,
#         c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))

In [ ]:
i_Ec = [6, 10, 50, 100, 1000]
delta = [-14.2,-17.5,-23.4,-21.8,-12.0]
delta_Ec_std = [4.54,6.39,12.1,13.3,19.1]

fig = figure('detection_rate_vs_mean_delta_energy'); clf()
ax1 = fig.add_subplot(111)


for k in range(3):
    ax1.errorbar(i_Ec,delta,yerr=delta_Ec_std, marker='D', ms=15, color='C1', markeredgecolor='k', linestyle='')
ax1.set_xlabel('Incident energy [eV]')
ax1.set_ylabel('Mean $\Delta E_c$ [meV]')

matplotlib.pyplot.grid(True, which="both")
# ax1.xaxis.set_major_locator(plt.MaxNLocator(25))
fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
savefig('detection_rate_vs_mean_delta_energy',dpi=300)

# ax.scatter(x,y,
#         marker='D',s=400,edgecolor='black',zorder = 1000,
#         c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))

In [ ]:
outfile = 'detection_rate_'+str(E0_GMol)+'eV'
# savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)
np.savez(outfile, E0_GMol=E0_GMol, detect_rate=detect_rate, mean_std_deltaEc=mean_std_deltaEc, points_and_coord=points_and_coord, Udc=Udc,Urf=Urf)

In [ ]:
outfile = 'detection_rate_10eV'
with load(outfile+'.npz') as data:
    a = data['mean_std_deltaEc']
print(a)

# Detection rate vs R/L

In [87]:
my_order = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,24,25,26,27,20,21,22,23]

R_moy = []
L_moy = []
std_RoverL = []

for k in range(28):
    mean_dim_nu = mean(dim_nu,axis = 1)[k][:]
    R_cond = divide(asarray(dim_nu)[k,:,0]+asarray(dim_nu)[k,:,1],2)
    L_cond = asarray(dim_nu)[k,:,2]
    
    R_moy.append( (mean_dim_nu[0]+mean_dim_nu[1])/2 )
    L_moy.append( mean_dim_nu[2] )
    std_RoverL.append(std(R_cond/L_cond,axis=0))
R_over_L = divide(R_moy,L_moy)

In [ ]:
fig = figure('detection_rate_vs_RoverL'); clf()
ax1 = fig.add_subplot(111)


for k in range(28//4):
    to_plot = my_order[k*4:(1+k)*4]
    ax1.errorbar(R_over_L[min(to_plot):max(to_plot)+1],multiply(detect_rate[min(to_plot):max(to_plot)+1],100),
                 xerr=std_RoverL[min(to_plot):max(to_plot)+1], marker='D', ms=15, color=cm(k/6), markeredgecolor='k', linestyle='--')
    
for i, txt in enumerate(points_and_coord):
    current_RoverL = R_moy[i]/L_moy[i]
    ax1.annotate(str(i), (current_RoverL, detect_rate[i]*100),color='xkcd:black',fontsize=10,
                ha='center',va='center',weight='bold',zorder = 1000)
    
ax1.set_xlabel(r'$\alpha$')
ax1.set_ylabel('Detection rate')

matplotlib.pyplot.grid(True, which="both")
# ax1.xaxis.set_major_locator(plt.MaxNLocator(25))
fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
# savefig('detection_rate_vs_mean_delta_energy',dpi=300)

# ax.scatter(x,y,
#         marker='D',s=400,edgecolor='black',zorder = 1000,
#         c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))

In [88]:
cm = pylab.get_cmap('rainbow')

In [89]:
fig = figure('detection_rate_vs_R2'); clf()
ax1 = fig.add_subplot(111)


for k in range(28//4):
    to_plot = my_order[k*4:(1+k)*4]
    ax1.errorbar(mean(asarray(dim_nu)[min(to_plot):max(to_plot)+1,:,0],axis=1)**2,multiply(detect_rate[min(to_plot):max(to_plot)+1],100),
                 marker='D', ms=15, color=cm(k/6), markeredgecolor='k', linestyle='--')
    
for i, txt in enumerate(points_and_coord):
    ax1.annotate(str(i),
                 ( mean(asarray(dim_nu)[i,:,0])**2, detect_rate[i]*100 ),
                 color='xkcd:black',fontsize=10,
                 ha='center',va='center',weight='bold',zorder = 1000)
    
ax1.set_xlabel(r'$R^2$')
ax1.set_ylabel('Detection rate')

matplotlib.pyplot.grid(True, which="both")
# ax1.xaxis.set_major_locator(plt.MaxNLocator(25))
fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
# savefig('detection_rate_vs_mean_delta_energy',dpi=300)

# ax.scatter(x,y,
#         marker='D',s=400,edgecolor='black',zorder = 1000,
#         c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))

In [489]:
fname = 'detection_rate_vs_L'
cm = pylab.get_cmap('cividis')
fig = plt.figure(num=fname,clear=True)
fig.set_size_inches(11.69,8.27)

gs1 = gridspec.GridSpec(1,1)
gs1.update(left=0.11, right=0.88, top=0.98, wspace=0, hspace = 0)
ax1 = plt.subplot(gs1[0])

gs2 = gridspec.GridSpec(1, 1)
gs2.update(left=0.89, right=0.92, top=0.98, wspace=0, hspace = 0)

for k in range(28//4):
    to_plot = my_order[k*4:(1+k)*4]
    ax1.errorbar(multiply(mean(asarray(dim_nu)[min(to_plot):max(to_plot)+1,:,2],axis=1), 1000),
                 multiply(detect_rate[min(to_plot):max(to_plot)+1],100),
                 xerr=std_RoverL[min(to_plot):max(to_plot)+1], marker='D', ms=15, color=cm(k/6), markeredgecolor='k', linestyle='--')
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=40))
for i, txt in enumerate(points_and_coord):
    current_RoverL = R_moy[i]/L_moy[i]
    ax1.annotate(str(i),
                 (mean(asarray(dim_nu)[i,:,2]), detect_rate[i]*100),color='xkcd:black',fontsize=10,
                ha='center',va='center',weight='bold',zorder = 1000)
    
ax1.set_xlabel(r'$L$ [µm]')
ax1.set_ylabel('Detection rate [%]')

axb = plt.subplot(gs2[0])
cmap = mpl.colors.ListedColormap([cm(k/6) for k in range(7)])
bounds = [0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
cb2 = mpl.colorbar.ColorbarBase(axb, cmap=cmap,
                                norm=norm,
                                ticks=np.arange(1,8), # np.arange(0.5,7.5)
                                spacing='proportional',
                                orientation='vertical')
cb2.set_label('$U_{DC}$ [V]')

ax1.grid(True, which="both")
# ax1.xaxis.set_major_locator(plt.MaxNLocator(25))
fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
# savefig('detection_rate_vs_mean_delta_energy',dpi=300)

# ax.scatter(x,y,
#         marker='D',s=400,edgecolor='black',zorder = 1000,
#         c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))

/home/adrian/.local/lib/python3.6/site-packages/matplotlib/figure.py:2267: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


In [108]:
x = np.array([1,2,3,4,5])
y = np.array([2,1,3,6,7])

cluster = np.array([1,1,1,2,2]) 

fig, ax = plt.subplots()

ax.scatter(x[cluster==1],y[cluster==1], marker='^')
ax.scatter(x[cluster==2],y[cluster==2], marker='s')

plt.show()

In [130]:
k = 2
to_plot = my_order[k*4:(1+k)*4]
x = multiply(mean(asarray(dim_nu)[min(to_plot):max(to_plot)+1,:,2],axis=1), 1000)
y = multiply(detect_rate[min(to_plot):max(to_plot)+1],100)
print(x,y)
for kk in range(4):
    print(x[cluster==kk+1],y[cluster==kk+1])

[219.83442633 227.99040842 236.3928712  243.606327  ] [  0.  30.  98. 100.]
[219.83442633] [0.]
[227.99040842] [30.]
[236.3928712] [98.]
[243.606327] [100.]


In [493]:
# with different marker for each RF in the same curve

fname = 'detection_rate_vs_L_custommarkers'
cm = pylab.get_cmap('cividis')
fig = plt.figure(num=fname,clear=True)
fig.set_size_inches(11.69,8.27)

gs1 = gridspec.GridSpec(1,1)
gs1.update(left=0.11, right=0.88, top=0.98, bottom=0.17, wspace=0, hspace = 0)
ax1 = plt.subplot(gs1[0])

gs2 = gridspec.GridSpec(1, 1)
gs2.update(left=0.89, right=0.92, top=0.98 , bottom=0.17, wspace=0, hspace = 0)

my_marker=['v','s','p','H']
my_label =[53.85, 59.23, 64.61, 70.00]
my_size =[225,150,225,225]

for k in range(28//4):
    to_plot = my_order[k*4:(1+k)*4]
    
    x = multiply(mean(asarray(dim_nu)[min(to_plot):max(to_plot)+1,:,2],axis=1), 1000)
    y = multiply(detect_rate[min(to_plot):max(to_plot)+1],100)
    cluster = np.array([1,2,3,4]) 

    for kk in range(4):
        ax1.scatter(x[cluster==kk+1],y[cluster==kk+1], marker=my_marker[kk],
                   s=my_size[kk], color=cm(k/6), edgecolor='k',zorder=1000)
    ax1.errorbar(x,y,
                 xerr=std_RoverL[min(to_plot):max(to_plot)+1],
                 marker='', color=cm(k/6), linestyle='--')

for kk in range(4):
        ax1.scatter(0,0, marker=my_marker[kk],
                   s=225, color='k', edgecolor='k',label=my_label[kk],zorder=0)
handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles[::-1], labels[::-1], title='$U_{RF}$', loc='lower right')
    
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=40))
    
ax1.set_xlabel(r'$\langle L\rangle / 2$ [µm]')
ax1.set_ylabel('Detection rate [%]')
ax1.set_xlim(118,412)

axb = plt.subplot(gs2[0])
cmap = mpl.colors.ListedColormap([cm(k/6) for k in range(7)])
bounds = [0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
cb2 = mpl.colorbar.ColorbarBase(axb, cmap=cmap,
                                norm=norm,
                                ticks=np.arange(1,8), # np.arange(0.5,7.5)
                                spacing='proportional',
                                orientation='vertical')
cb2.set_label('$U_{DC}$ [V]')

ax1.grid(True, which="both")
# ax1.xaxis.set_major_locator(plt.MaxNLocator(25))
fig.set_size_inches(11.69, 6.1)
# savefig('detection_rate_vs_mean_delta_energy',dpi=300)

# ax.scatter(x,y,
#         marker='D',s=400,edgecolor='black',zorder = 1000,
#         c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))

In [494]:
savefig(fname+'.png',dpi=300)
savefig(fname+'.eps',dpi=600,format='eps')

In [ ]:
fig = figure('L_vs_R2'); clf()
ax1 = fig.add_subplot(111)


for k in range(28//4):
    to_plot = my_order[k*4:(1+k)*4]
    ax1.errorbar(mean(asarray(dim_nu)[min(to_plot):max(to_plot)+1,:,0],axis=1)**2,mean(asarray(dim_nu)[min(to_plot):max(to_plot)+1,:,2],axis=1),
                 xerr=0, marker='', ms=15, linestyle='--', label=my_order[k]+1)
ax1.legend(title='Udc')
    
for i, txt in enumerate(points_and_coord):
    print(i,txt)
    ax1.annotate(str(i),
                 (mean(asarray(dim_nu)[i,:,0])**2,mean(asarray(dim_nu)[i,:,2])),
                 color='xkcd:black',fontsize=10,
                 ha='center',va='center',weight='bold',zorder = 5)
    ax1.scatter(mean(asarray(dim_nu)[i,:,0])**2,mean(asarray(dim_nu)[i,:,2]),
                 marker='D',edgecolor='k', s = 15**2, c=cm(detect_rate[my_order[i]]), zorder = 3)
    
ax1.set_xlabel(r'$L$')
ax1.set_ylabel('$R^2$')

matplotlib.pyplot.grid(True, which="both")
# ax1.xaxis.set_major_locator(plt.MaxNLocator(25))
fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
# savefig('detection_rate_vs_mean_delta_energy',dpi=300)

# ax.scatter(x,y,
#         marker='D',s=400,edgecolor='black',zorder = 1000,
#         c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))

# Ion density

In [ ]:
print(shape(dim_nu),shape(Gmol_data[0]),shape(detect_rate))
print(mean(dim_nu[3][:][:],axis=0),std(dim_nu[3][:][:],axis=0))
print(mean(dim_nu[12][:][:],axis = 0),mean(dim_nu[3][:][1]))

In [ ]:
my_order = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,24,25,26,27,20,21,22,23]

cloud_density = zeros((28,100))
for k in range(28):
    for l in range(100):
        R_moy = ( dim_nu[k][l][0] + dim_nu[k][l][1] ) / 2 *1e-3
        L_moy = dim_nu[k][l][2] *1e-3
    #     print(k,R_moy,L_moy)
        V = 4/3*pi*R_moy**2 * L_moy
        cloud_density[my_order[k],l] = N_ions / V
std_cloud_density = [std(x) for _,x in enumerate(cloud_density)]

cloud_density_formula = zeros(28)
for k,l in enumerate(list(points_and_coord)):
    my_udc = Udc[int(l[2:4])]
    my_urf = Urf[int(l[7:])]
    print(l,my_order[k],my_udc,my_urf)
    q = 4 * C_e * my_urf / (m_Ca*r0**2*Omega**2)
    omegaz = sqrt( 2 * C_e * my_udc / (m_Ca*d0**2) )
    omegar_2 = Omega**2 / 4 * ( q**2/2 - 0.5*(omegaz/Omega)**2 )
    cloud_density_formula[my_order[k]] = 2*m_Ca*eps0/C_e**2 * omegar_2

In [ ]:
cloud_density,cloud_density_formula

In [ ]:
fig = figure('detection_rate_vs_ion_density',clear=True)
ax1 = fig.add_subplot(111)
fig.set_size_inches(11.69, 8.27)

for k in range(28//4):
    to_plot = my_order[k*4:(1+k)*4]
    ax1.errorbar(mean(cloud_density,axis=1)[min(to_plot):max(to_plot)],
                 multiply(detect_rate,100)[min(to_plot):max(to_plot)],
                 xerr=std_cloud_density[min(to_plot):max(to_plot)],
             marker='D', ms=10, color=cm(k/6), markeredgecolor='k', linestyle=':',label=k+1)
ax1.set_xlabel('cloud_density [ion/m^3]')
ax1.set_ylabel('detect_rate [%]')
ax1.legend(title=r'$U_{DC}$')

for i, txt in enumerate(points_and_coord):
    ax1.annotate(str(i), (mean(cloud_density,axis=1)[i], detect_rate[i]*100+2.6),
    color='xkcd:black',fontsize=10, # cm(1-(i//4)/6)
    ha='center',va='center',weight='bold',zorder = 1000)

matplotlib.pyplot.grid(True, which="both")
# ax1.xaxis.set_major_locator(plt.MaxNLocator(25))

plt.tight_layout()
# savefig('detection_rate_vs_ion_density',dpi=300)

# ax.scatter(x,y,
#         marker='D',s=400,edgecolor='black',zorder = 1000,
#         c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))

In [ ]:
fig = figure('detection_rate_vs_ion_density_formula'); clf()
ax1 = fig.add_subplot(111)

ax1.plot(cloud_density_formula,multiply(detect_rate,100), marker='D', ms=15, color='C1', markeredgecolor='k', linestyle='')
ax1.set_xlabel('cloud_density_formula [ion/m^3]')
ax1.set_ylabel('detect_rate [%]')

for i, txt in enumerate(points_and_coord):
    ax1.annotate(str(i), (cloud_density_formula[i], detect_rate[i]*100),color='xkcd:black',fontsize=10,
                ha='center',va='center',weight='bold',zorder = 1000)

matplotlib.pyplot.grid(True, which="both")
# ax1.xaxis.set_major_locator(plt.MaxNLocator(25))
fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
savefig('detection_rate_vs_ion_density_formula',dpi=300)

# ax.scatter(x,y,
#         marker='D',s=400,edgecolor='black',zorder = 1000,
#         c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))

In [ ]:
R_over_L[min(to_plot):max(to_plot)+1],mean(cloud_density,axis=1)[min(to_plot):max(to_plot)]

In [ ]:
fig = figure('density_vs_RoverL'); clf()
ax1 = fig.add_subplot(111)


for k in range(28//4):
    to_plot = my_order[k*4:(1+k)*4]
    ax1.errorbar(R_over_L[min(to_plot):max(to_plot)+1],mean(cloud_density,axis=1)[min(to_plot):max(to_plot)+1],
                 marker='', ms=15, color=cm(k/6), linestyle='--',label=k+1)
ax1.legend(title='Udc')
for i, txt in enumerate(points_and_coord):
    current_RoverL = R_moy[i]/L_moy[i]
    ax1.annotate(str(i), (current_RoverL, mean(cloud_density,axis=1)[i]),color='xkcd:black',fontsize=10,
                ha='center',va='center',weight='bold',zorder = 1000)
    ax1.scatter(R_over_L[i],mean(cloud_density,axis=1)[i],
             marker='D',edgecolor='k', s = 15**2, c=cm(detect_rate[my_order[i]]), zorder = 3)
    
ax1.set_xlabel(r'$\alpha$')
ax1.set_ylabel('Cloud density')

matplotlib.pyplot.grid(True, which="both")
# ax1.xaxis.set_major_locator(plt.MaxNLocator(25))
fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
# savefig('detection_rate_vs_mean_delta_energy',dpi=300)

# ax.scatter(x,y,
#         marker='D',s=400,edgecolor='black',zorder = 1000,
#         c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))

# Saving data

In [ ]:
outfile = 'GMol_info_'+str(E0_GMol)+'eV'
# savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)
np.savez(outfile, deltaEc=Gmol_data[0], SNR=Gmol_data[2], points_and_coord=points_and_coord, Udc=Udc,Urf=Urf)

In [ ]:
dirname_hc = str(data0[0][0])+'/'
str_eta = '0.15D-19'
datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T = \
load_Temp_init_bin_Lan(dirname_hc+datainit_hc,flag_print=0)

shape(save_T)

In [ ]:
to_plot = 0
str_eta = '0.15D-19'
for l in range(0,7): # b
    for k in range(0,4): # a
        plot_point = to_plot
        all_stat_time = []
        all_stat_Temp = []
                
        for plot_try in range(100):
            dirname_hc = str(data0[plot_point][plot_try])+'/'
            # Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
            datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
            N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T = \
            load_Temp_init_bin_Lan(dirname_hc+datainit_hc,flag_print=0)
            # Get T and fluo PM
            onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and 'SimuType' in f and '.dat' in f]
            ta, te, ti = plot_T_and_PM_InitQ_Inje_Evol(dirname_hc,sort(onlyfiles)[2].strip('.dat')[4:],flag_plot = 0,fig_name = 'Temp_allpoints')
            
            tt_Lan = linspace(0,dt_j_save_next*j_save,j_save)
            tt_md = ta[0]
            T_aux = te[0]
            
            tt = concatenate((tt_Lan,tt_md))
            T_CM = concatenate((save_T[:,1:4],T_aux),axis=0)
            T_all = concatenate((save_T[:,4:],T_aux),axis=0)
            
            all_stat_time.append(tt)
            all_stat_Temp.append(T_all)
            
            the_current_cond = list(dict.fromkeys(points_and_coord))[plot_point]
            outfile = 'T_cloud_'+str(the_current_cond)+'_'+str(E0_GMol)+'eV'
            np.savez( outfile, time=all_stat_time, Temperature=all_stat_Temp, condition=the_current_cond) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)
            
        to_plot += 1
        if to_plot == 28:
            break
    if to_plot == 28:
        break

In [ ]:
# load check
outfile = 'GMol_info_50eV'
with load(outfile+'.npz') as data:
    a = data['Udc']
print(a)